## Day 20: Particle Swarm
    
http://adventofcode.com/2017/day/20

### Part 1

The answer here is the particle with the smallest acceleration magnitude. If two particles with different accelerations start from completely arbitrary finite locations and velocities, then at some point the particle with the higher acceleration will have higher velocity and continue to have higher velocity, so then at some possibly later point will be diverging further and faster from the origin. 

The input format is a bit awkward, so I've used Emacs to turn it into csv.

In [9]:
def split_by_length(xs, length):
    l_xs = list(xs)
    return [l_xs[i:i+length] for i in range(0, len(l_xs), length)]

        
with open('input.csv') as f:
    particles = [split_by_length(map(int, line.strip().split(',')), 3) 
                 for line in f]
    
particles[0]

[[-717, -4557, 2578], [153, 21, 30], [-8, 8, -7]]

Using Manhattan vectors is a bit odd, but I can't see why it wouldn't work for this. The particle having acceleration with the largest Manhattan magnitude will still move further away than others when using Manhattan distance as a measure of distance.

In [17]:
def manhattan_magnitude(v):
    return sum(abs(v_i) for v_i in v)

def acceleration_magnitude(p):
    return manhattan_magnitude(p[2])

acceleration_magnitude(particles[0])

23

In [18]:
min(range(len(particles)), key=lambda i: acceleration_magnitude(particles[i]))

115

In [19]:
particles[115]

[[-129, 1190, -665], [3, -82, 41], [1, 0, 1]]

That looks small, let's give it a go.

It's the wrong answer. What am I missing? It is quite early.

In [21]:
sorted(particles, key=acceleration_magnitude)[:10]

[[[-129, 1190, -665], [3, -82, 41], [1, 0, 1]],
 [[528, -1788, -1536], [3, 77, 65], [-2, 0, 0]],
 [[692, -175, -1125], [-38, 19, 49], [0, -1, 1]],
 [[409, 864, -1345], [1, -28, 32], [-1, 0, 1]],
 [[-1085, -561, 1531], [41, -8, -56], [0, 2, 0]],
 [[1119, -358, -696], [-83, 20, 46], [2, 0, -1]],
 [[-940, 1847, 1177], [45, -71, -30], [0, -1, -2]],
 [[1720, 726, -1414], [-33, -28, 64], [-2, 0, -1]],
 [[-932, 143, -1423], [50, -10, 107], [1, 0, -2]],
 [[-2370, 3605, -22], [91, -112, 15], [0, -2, -1]]]

Of course, I need to distinguish between particles with the same acceleration. Use velocity as the tie-break. For two particles with the same acceleration one with the higher initial velocity will always have a higher velocity and diverge further from the origin.

In [23]:
def velocity_magnitude(p):
    return manhattan_magnitude(p[1])

min(range(len(particles)), 
    key= lambda i: (acceleration_magnitude(particles[i]), 
                    velocity_magnitude(particles[i])))

344

In [24]:
particles[344]

[[409, 864, -1345], [1, -28, 32], [-1, 0, 1]]